In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
import _pickle as cPickle
import numpy as np
import os

CIFAR_DIR = "/content/drive/My Drive/Colab Notebooks/cifar-10-batches-py"
print(os.listdir(CIFAR_DIR)) 

['readme.html', 'batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3', '.ipynb_checkpoints']


In [0]:
def load_data(filename):
    """read data from data file."""
    
    # 注： 在load数据时，通过添加encoding='bytes'，解决'ascii' codec can't decode byte 0x8b in position 6: ordinal not in range(128)
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.Dataset  同样可以实现CifarData的效果
class CifarData:
    def __init__(self, filenames, need_shuffle): # need_shuffle: 指是否需要打乱样本的顺序，越无序，训练的结果越好
        all_data = []
        all_labels = []
        for filename in filenames:
            datas, labels = load_data(filename)
            all_data.append(datas)
            all_labels.append(labels)
        
        self._data = np.vstack(all_data)
         
        # 对数据做归一化处理，像素值都是在0-255之间，故做如下处理，将其归一化在（-1，1）之间 # 提高训练的准确率
        # 对比，没有self._data = self._data / 127.5 - 1 和有此语句的准确率
        # 因为像素值是在 0——255之间，所以，/127.5，其值在0-2之间，再减去1，则确保元素的值在（-1，1）之间。
        # 因为 sigmoid函数若数据偏向一方，梯度会消失，归一化之后将不再会出现这种问题
        self._data = self._data / 127.5 - 1 
        self._labels = np.hstack(all_labels)
        
        
        # print("self._data.shape")
        # print(self._data.shape)
        # print("self._labels.shape")
        # print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0  # self._indicator 用来分batch的时候用
        
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

# batch_data, batch_labels = train_data.next_batch(500)
# print('batch_data')
# print(batch_data)
# print('batch_labels')
# print(batch_labels)

In [4]:
x = tf.placeholder(tf.float32, [None, 3072]) # 32*32*3 = 3072 
y = tf.placeholder(tf.int64, [None])

# 由于卷积神经网络输入的是图片，所以输入x应该是具有3通道的图片格式
# 图像大小： 32*32*3
x_image = tf.reshape(x, [-1, 3, 32, 32]) # -1，表示的是样本的个数
x_image = tf.transpose(x_image, perm=[0,2,3,1]) #交换通道

# conv1:神经元图， feature_map， 输出图像
# 过卷积层图像大小没有变化
# 输入图像的size：32*32
conv1_1 = tf.layers.conv2d(x_image, 
                           32, # output channel number
                           (3,3), # kernel size
                           padding = 'same', # 是否补0，若为'same'，则说明输入图像的大小和输出图像的大小相同
                           activation = tf.nn.relu,
                           name = 'conv1_1') # 给本层做一个命名，打印图像时显示，以便对每一层作区分

conv1_2 = tf.layers.conv2d(conv1_1, 
                           32, # output channel number
                           (3,3), # kernel size
                           padding = 'same', # 是否补0，若为'same'，则说明输入图像的大小和输出图像的大小相同
                           activation = tf.nn.relu,
                           name = 'conv1_2') # 给本层做一个命名，打印图像时显示，以便对每一层作区分


# 过池化层，kernel size为2， 图像变为以前的二分之一
pooling1 = tf.layers.max_pooling2d(conv1_2,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool1')

# 输入图像的size：16*16
conv2_1 = tf.layers.conv2d(pooling1, 
                           32, # output channel number
                           (3,3), # kernel size
                           padding = 'same', # 是否补0，若为'same'，则说明输入图像的大小和输出图像的大小相同
                           activation = tf.nn.relu,
                           name = 'conv2_1')

conv2_2 = tf.layers.conv2d(conv2_1, 
                           32, # output channel number
                           (3,3), # kernel size
                           padding = 'same', # 是否补0，若为'same'，则说明输入图像的大小和输出图像的大小相同
                           activation = tf.nn.relu,
                           name = 'conv2_2')

pooling2 = tf.layers.max_pooling2d(conv2_2,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool2')

# 输入图像的size：8*8
conv3_1 = tf.layers.conv2d(pooling2, 
                           32, # output channel number
                           (3,3), # kernel size
                           padding = 'same', # 是否补0，若为'same'，则说明输入图像的大小和输出图像的大小相同
                           activation = tf.nn.relu,
                           name = 'conv3_1')

conv3_2 = tf.layers.conv2d(pooling2, 
                           32, # output channel number
                           (3,3), # kernel size
                           padding = 'same', # 是否补0，若为'same'，则说明输入图像的大小和输出图像的大小相同
                           activation = tf.nn.relu,
                           name = 'conv3_2')

pooling3 = tf.layers.max_pooling2d(conv3_2,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pooling3')

# 输入图像的size：4*4*32
flatten= tf.layers.flatten(pooling3) # 将其展成 [None, 4*4*32] 
y_ = tf.layers.dense(flatten, 10)

# mean square loss
# p_y = tf.nn.softmax(y_) # p_y [[0.1, 0.01, 0.9 ......0.03], [], [], ...[]]
# y_one_hot = tf.one_hot(y, 10, dtype = tf.float32) # y [2,......] y_one_hot [[0, 0, 1, ......0], [], [], ...[]]
# loss = tf.reduce_mean(tf.square(y_one_hot - p_y))

# Cross entropy loss
# y_ -> softmax
# y -> one_hot
# loss = ylogy_
loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits= y_)

# index
y_predict = tf.argmax(y_, 1)

#[1, 0, 1, 1, 1, 0, 0, 0]
correct_prediction = tf.equal(y_predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

# tf,name_scope: A context manager for use when defining a Python op.
# tf.train.AdamOptimizer: Optimizer that implements the Adam algorithm.
# ? Adam algorithm

# tf.train.AdamOptimizer.minimize
# Add operations to minimize loss by updating var_list.
# This method simply combines calls compute_gradients() and apply_gradients(). 
# If you want to process the gradient before applying them call compute_gradients() and apply_gradients() 
# explicitly instead of using this function
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
    

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
init = tf.global_variables_initializer()
batch_size = 500
train_steps = 5000
test_steps = 20

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps): 
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict = {
                x: batch_data,
                y: batch_labels})
        
        if i % 500 == 0:
            print('[Train] step: %d, loss: %4.5f, acc: %4.5f'\
                 % (i, loss_val, acc_val))
        
        # 在测试集上评测的代码
        if i % 500 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels})
                
                all_test_acc_val.append(test_acc_val)
            
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' % (i, test_acc))

[Train] step: 0, loss: 2.31498, acc: 0.11600
[Test] Step: 0, acc: 0.12810
